In [26]:
import api
import pandas as pd
df = api.get_merged_dataframe('14L', '5.1')
#14L to select Manchester CCG and 5.1 to select Antibacterial group
df.head()

,actual_cost,ccg,date,items,quantity,row_id,row_name,setting,total_list_size,items_per_patient
0,2856.64,14L,2014-10-01,417,18650,P84004,FIVE OAKS FAMILIY PRACTICE,4,8819,0.047284
1,919.79,14L,2014-10-01,203,8165,P84005,THE VALLANCE CENTRE,4,7223,0.028105
2,2063.74,14L,2014-10-01,357,17084,P84009,AILSA CRAIG MEDICAL CENTRE,4,9342,0.038215
3,1873.21,14L,2014-10-01,423,17154,P84010,THE BORCHARDT MEDICAL CENTRE,4,11108,0.038081
4,2992.22,14L,2014-10-01,644,28069,P84012,NORTHENDEN GROUP PRACTICE,4,11028,0.058397


In [8]:
def get_min_max_practice_from_dataframe(df):
    means={}
    for practice in df['row_name'].unique():
        practice_df = df[df.row_name == practice]
        average = practice_df["items_per_patient"].mean()
        means[practice] = average 
    minimum = min(means, key=means.get)
    maximum = max(means, key=means.get)
    output = {
        "minimum": minimum,
        "maximum": maximum
    }
    return output
    

means=get_min_max_practice_from_dataframe(df)

In [39]:
def plot_min_max_mean(df):
    minmax = get_min_max_practice_from_dataframe(df)
    mean = df['items_per_patient'].groupby(df['date']).mean()
    mean = pd.DataFrame({'date': mean.index, 'mean': mean.values})
    minimum = df[df.row_name == minmax['minimum']]
    maximum = df[df.row_name == minmax['maximum']]
    df = mean.merge(minimum, maximum)
    return df 

minmaxdf = plot_min_max_mean(df)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [21]:
df['items_per_patient'].groupby(df['date']).min()

date
2014-10-01    0.015320
2014-11-01    0.014856
2014-12-01    0.005571
2015-01-01    0.004632
2015-02-01    0.000463
2015-03-01    0.001390
2015-04-01    0.001475
2015-05-01    0.003912
2015-06-01    0.001422
2015-07-01    0.003632
2015-08-01    0.001453
2015-09-01    0.001090
2015-10-01    0.002228
2015-11-01    0.015596
2015-12-01    0.017282
2016-01-01    0.020725
2016-02-01    0.021789
2016-03-01    0.023735
2016-04-01    0.017516
2016-05-01    0.012546
2016-06-01    0.016720
2016-07-01    0.018692
2016-08-01    0.015172
2016-09-01    0.018098
2016-10-01    0.006887
2016-11-01    0.003562
2016-12-01    0.000950
2017-01-01    0.020318
2017-02-01    0.017707
2017-03-01    0.020237
2017-04-01    0.014445
2017-05-01    0.015478
2017-06-01    0.014518
2017-07-01    0.009578
2017-08-01    0.009838
2017-09-01    0.007272
2017-10-01    0.013903
2017-11-01    0.016348
2017-12-01    0.014250
2018-01-01    0.019386
2018-02-01    0.011092
2018-03-01    0.010826
2018-04-01    0.011613
2018-0

In [16]:
joescoo['mean'].idxmin()

'THE VALLANCE CENTRE'

In [17]:
means

{'minimum': 'THE VALLANCE CENTRE', 'maximum': 'HAWTHORN MC'}